In [199]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm_notebook

In [423]:
# 데이터 불러오기
total_data = pd.read_csv('data_te.csv', encoding='ANSI')
total_data.head()

,DATE,REVIEW,star
0,2021-03-10,몇달만에 접속했는데. 서버도 다 통합 되어있고 내가 키우던 캐릭이 하나도 없이. 나...,3
1,2020-03-16,오랜만에 복귀하려고 설치해봤습니다. 하지만 처음 서버 선택 후 캐릭터선택창에 진입시...,1
2,2021-03-01,불편한 점 하나 얘기하자면 음악 들으면서 게임하고플 때 소리 다 줄이려면 일일이 볼...,4
3,2021-03-08,게임 처음 설치 후 다운로드가 현재 8시간 동안 진행중인데도 불구하고 아직 1.2G...,3
4,2021-03-03,점검하고 나서 들어갔는데 계정이 다 날아갔어요 어떻게 된거죠? 최대한 빨리 복구해주...,3


In [424]:
# 리뷰 입력
a = input('리뷰를 입력해주세요 : ')

리뷰를 입력해주세요 : 너무 재미있습니다. 어렵지만 게임이 좀 재밌어서 계속 하게 돼요 유일하게 많이 하는 게임이에요!


In [425]:
# 4~5 별점 data만 남기고 나머지는 0
total_data['label'] = np.select([total_data.star > 3], [1], default=0)
total_data[:5]

,DATE,REVIEW,star,label
0,2021-03-10,몇달만에 접속했는데. 서버도 다 통합 되어있고 내가 키우던 캐릭이 하나도 없이. 나...,3,0
1,2020-03-16,오랜만에 복귀하려고 설치해봤습니다. 하지만 처음 서버 선택 후 캐릭터선택창에 진입시...,1,0
2,2021-03-01,불편한 점 하나 얘기하자면 음악 들으면서 게임하고플 때 소리 다 줄이려면 일일이 볼...,4,1
3,2021-03-08,게임 처음 설치 후 다운로드가 현재 8시간 동안 진행중인데도 불구하고 아직 1.2G...,3,0
4,2021-03-03,점검하고 나서 들어갔는데 계정이 다 날아갔어요 어떻게 된거죠? 최대한 빨리 복구해주...,3,0


In [426]:
print('전체 데이터의 총 개수: {}'.format(len(total_data)))

전체 데이터의 총 개수: 54074


In [427]:
print("부정 리뷰 개수: {}".format(total_data['label'].value_counts()[0]))
print("긍정 리뷰 개수: {}".format(total_data['label'].value_counts()[1]))

부정 리뷰 개수: 27602
긍정 리뷰 개수: 26472


In [428]:
total_word_counts = total_data['REVIEW'].astype(str).apply(lambda x:len(x.split(' ')))

In [429]:
print('리뷰 단어 개수 최대 값: {}'.format(np.max(total_word_counts)))
print('리뷰 단어 개수 최소 값: {}'.format(np.min(total_word_counts)))
print('리뷰 단어 개수 평균 값: {:.2f}'.format(np.mean(total_word_counts)))
print('리뷰 단어 개수 표준편차: {:.2f}'.format(np.std(total_word_counts)))
print('리뷰 단어 개수 중간 값: {}'.format(np.median(total_word_counts)))
# 사분위의 대한 경우는 0~100 스케일로 되어있음
print('리뷰 단어 개수 제 1 사분위: {}'.format(np.percentile(total_word_counts, 25)))
print('리뷰 단어 개수 제 3 사분위: {}'.format(np.percentile(total_word_counts, 75)))

리뷰 단어 개수 최대 값: 99
리뷰 단어 개수 최소 값: 1
리뷰 단어 개수 평균 값: 14.90
리뷰 단어 개수 표준편차: 14.70
리뷰 단어 개수 중간 값: 10.0
리뷰 단어 개수 제 1 사분위: 5.0
리뷰 단어 개수 제 3 사분위: 19.0


In [430]:
qmarks = np.mean(total_data['REVIEW'].astype(str).apply(lambda x: '?' in x)) # 물음표가 구두점으로 쓰임
fullstop = np.mean(total_data['REVIEW'].astype(str).apply(lambda x: '.' in x)) # 마침표
                  
print('물음표가있는 질문: {:.2f}%'.format(qmarks * 100))
print('마침표가 있는 질문: {:.2f}%'.format(fullstop * 100))

물음표가있는 질문: 17.15%
마침표가 있는 질문: 42.82%


In [431]:
import re
import json
from konlpy.tag import Okt
from tqdm import tqdm_notebook
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [432]:
total_data['REVIEW'] = total_data['REVIEW'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [433]:
total_data['REVIEW'].replace('', np.nan, inplace=True)
total_data = total_data.dropna(how='any')

print(total_data.isnull().sum())

DATE      0
REVIEW    0
star      0
label     0
dtype: int64


In [434]:
def define_stopwords(path):    
    SW = set()   
    with open(path) as f:
        for word in f:
            word = re.sub("\n", "", word)
            SW.add(word)            
    return SW
stop_words = define_stopwords("stopwords.txt")
# stop_words 

In [435]:
def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^a-zA-Z가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", review)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    result = []
    word_s = okt.pos(review_text, norm=True, stem=True)
    for n, h in word_s:
        if not (h in ['Noun', 'Verb ', 'Adjective']): continue
        if h == 'Punctuation' and h2 == 'Number': continue
        result.append(n)
    
    if remove_stopwords:        
        # 불용어 제거(선택적)
        word_review = [token for token in result if not token in stop_words]
        # 단어 리스트를 공백을 넣어서 하나의 글로 합친다.
        clean_review = ' '.join(word_review)
         
    return clean_review

In [436]:
clean_total_review = []

for review in tqdm_notebook(total_data['REVIEW']):
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_total_review.append(preprocessing(review, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        clean_total_review.append([])  #string이 아니면 비어있는 값 추가

In [437]:
# 입력한 리뷰를 지정 함수에 넣고 전처리 진행
review1 = preprocessing(a, okt, remove_stopwords=True, stop_words=stop_words)
review1
# 전처리 안된 리뷰와 전처리 된 리뷰를 나눠서 저장
n = review1

In [438]:
import string # 특수문자
import re
import warnings # 경고 알림 제거
import pickle
import nltk
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [439]:
print(len(clean_total_review))
print(clean_total_review[0:5])
print(n)

54050
['달 접속 서버 통합 내 캐릭 하나 나 신규 유저 취급 요 게임 내의 문의 해도 답 없다 다시키 케릭', '만 복귀 설치 처음 서버 선택 후 캐릭터 선택 창 진입 시 처음 혼잡하다 서버 문제 줄 알 여러 번 시도 실패 후 타 서버 새롭다 캐릭터 생 뒤 다시 첫 화면 역시 캐릭터 선택 창 진입 오류 수정 부탁드리다', '불편하다 점 하나 얘기 음악 게임 때 소리 일일이 볼륨 게 불편하다 음소거 기능 있다 좋다', '게임 처음 설치 후 다운로드 현재 시간 동안 진행중 도 불구 인터넷 상태 호한 데 왜 이 게임 이렇다', '점검 계정 어떻다 된거 최대한 복구 게임 재밌다 있다']
재미있다 어렵다 게임 좀 재밌다 계속 유일하다 게임


In [440]:
# 자카드 유사도를 사용자 함수로 표현
def Jaccard_similarity(doc1, doc2):
    doc1 = set(doc1)
    doc2 = set(doc2)
    return len(doc1 & doc2) / len(doc1 | doc2)

In [447]:
# 입력된 리뷰와 리뷰 데이터를 자카드 함수에 입력
result = []
for i in tqdm_notebook(range(len(clean_total_review))):
    k = Jaccard_similarity(a, clean_total_review[i])
    result.append(k)

In [442]:
len(result)

54050

In [443]:
result[0:5]

[0.17073170731707318,
 0.046875,
 0.21212121212121213,
 0.10869565217391304,
 0.3333333333333333]

In [448]:
m = max(result)
m

0.35714285714285715

In [449]:
r = result.index(max(result))
r

28356

In [450]:
df_total['REVIEW'][r]    # 밑의 total_data의 리뷰와 df_total의 리뷰가 다르다...

'초반 괜찮다 갑자기 게임 인증 거 거만 좋다'

In [451]:
total_data['REVIEW'][r]

'옛날이 더 좋았다 옛날에는 정말 순수하게 재밌는 게임이었다 요즘은 너무 복잡해서 뭐가 뭔지 모르겠다'